In [8]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    # Extract the ratio parts
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    
    # Calculate risk amount based on mean candle size
    risk_amount = mean_candle_size * risk_part
    
    # Calculate reward amount based on risk reward ratio
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        # Calculate stop loss and take profit prices for buy order
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        # Calculate stop loss and take profit prices for sell order
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
mt5.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_D1  # Hourly timeframe

# Define start date and end date
start_date = datetime(2021, 9, 15, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 6, 22, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
ohlc_data = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close'] 

# Load predictions from CSV file
df_pred = pd.read_csv('predictlstmUSDCAD_D1Buy.csv', index_col=0)

# Ensure the index of the prediction dataframe is of datetime type
df_pred.index = pd.to_datetime(df_pred.index)

# Define a simple strategy based on predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0088
    risk_reward_ratio = '3:4'  # Default value

    def init(self):
        setattr(self, 'risk_reward_ratio', self.risk_reward_ratio)  # Set the risk_reward_ratio attribute
        print(self.risk_reward_ratio)

    def next(self):
        current_time = self.data.index[-1]
        
        # Check if the current time is in the predictions index
        if current_time in df_pred.index:
            prediction = df_pred.loc[current_time, 'prediction']
            entry_price = self.data.Close[-1]
            
            if prediction == 1:
                order_type = 'buy'
                sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
                self.buy(size=10000, sl=sl_price, tp=tp_price)  # Buy if prediction is 1

# Create and run backtest with the SimpleStrategy
bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
output = bt.run()
print(output)

""" risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4', '2:2', '2:3', '2:4', '2:5']

# Optimize the strategy for SL/TP ratios
stats = bt.optimize(
    risk_reward_ratio=risk_reward_ratios,
    maximize='Equity Final [$]',
    constraint=lambda p: ':' in p.risk_reward_ratio
)

# Print optimized strategy stats
print(f"Best risk_reward_ratio: {stats._strategy.risk_reward_ratio}")
print(stats) """

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Plot the results of the optimized strategy
bt.plot()


3:4
Start                     2021-09-15 00:00:00
End                       2024-06-21 00:00:00
Duration                   1010 days 00:00:00
Exposure Time [%]                   93.611111
Equity Final [$]                  47974.26414
Equity Peak [$]                   52454.59513
Return [%]                         379.742641
Buy & Hold Return [%]                8.471354
Return (Ann.) [%]                    73.12226
Volatility (Ann.) [%]              121.377719
Sharpe Ratio                         0.602436
Sortino Ratio                        1.939825
Calmar Ratio                         1.325672
Max. Drawdown [%]                  -55.158634
Avg. Drawdown [%]                   -8.163752
Max. Drawdown Duration      234 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                  292
Win Rate [%]                        64.726027
Best Trade [%]                       2.927515
Worst Trade [%]                     -2.627337
Avg. Trade [%]                

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p4771', ...)

In [6]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    else:
        raise ValueError("order_type must be 'buy'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_D1  # Changed timeframe to D1

# Define start date and end date
start_date = datetime(2023, 1, 25, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 6, 17, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print("Failed to retrieve data")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions from CSV files
df_predbuy = pd.read_csv('predictUSDCAD_D1Buy.csv', index_col=0)  # Changed to D1 buy predictions
df_predbuy.index = pd.to_datetime(df_predbuy.index)

# Define a simple strategy based on buy predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0090
    risk_reward_ratiobuy = '2:3'

    def init(self):
        self.risk_reward_ratiobuy = self.risk_reward_ratiobuy

    def next(self):
        current_time = self.data.index[-1]
        
        # Get buy prediction for the current time
        buy_prediction = df_predbuy.loc[current_time, 'prediction'] if current_time in df_predbuy.index else 0
        
        # Place a buy order if the buy prediction is 1
        if buy_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiobuy, order_type, self.mean_candle_size)
            self.buy(size=10000, sl=sl_price, tp=tp_price)

# List of risk-reward ratios to test
risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4', '2:2', '2:3', '2:4', '2:5']

# DataFrame to store results for each risk-reward ratio
results_columns = ['Equity Final [$]', 'Equity Peak [$]', 'Return [%]', 'Max. Drawdown [%]',
                   'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration', 'Win Rate [%]']
results_df = pd.DataFrame(index=risk_reward_ratios, columns=results_columns)

# Run backtest for each risk-reward ratio
for buy_ratio in risk_reward_ratios:
    print(f"Testing risk-reward ratio: buy={buy_ratio}")
    bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
    stats = bt.run(risk_reward_ratiobuy=buy_ratio)
    
    # Store the results in the DataFrame
    results_df.loc[buy_ratio, 'Equity Final [$]'] = stats['Equity Final [$]']
    results_df.loc[buy_ratio, 'Equity Peak [$]'] = stats['Equity Peak [$]']
    results_df.loc[buy_ratio, 'Return [%]'] = stats['Return [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown [%]'] = stats['Max. Drawdown [%]']
    results_df.loc[buy_ratio, 'Avg. Drawdown [%]'] = stats['Avg. Drawdown [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown Duration'] = stats['Max. Drawdown Duration']
    results_df.loc[buy_ratio, 'Avg. Drawdown Duration'] = stats['Avg. Drawdown Duration']
    results_df.loc[buy_ratio, 'Win Rate [%]'] = stats['Win Rate [%]']

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Print the results matrix
print("\nResults matrix:")
print(results_df)

# Optionally plot the last backtest results
bt.plot()


FileNotFoundError: [Errno 2] No such file or directory: 'predictUSDCAD_D1Buy.csv'

In [4]:
results_df_sorted = results_df.sort_values(by='Max. Drawdown Duration', ascending=True)

In [5]:
results_df_sorted

,Equity Final [$],Equity Peak [$],Return [%],Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,Win Rate [%]
2:2,164023.8935,170671.2831,1540.238935,-50.791284,-11.551471,126 days 00:00:00,17 days 00:00:00,75.144509
2:3,193542.6379,202323.0275,1835.426379,-97.962487,-18.373373,128 days 00:00:00,24 days 00:00:00,67.901235
2:4,245415.1344,259697.524,2354.151344,-97.962487,-17.713884,143 days 00:00:00,22 days 00:00:00,66.463415
2:5,231417.9673,250171.3689,2214.179673,-97.962487,-27.580422,181 days 00:00:00,33 days 00:00:00,59.876543
1:1,27926.458,30057.89,179.26458,-74.7567,-14.553983,201 days 00:00:00,37 days 00:00:00,54.913295
1:3,65092.1525,71514.5421,550.921525,-89.169987,-22.251693,218 days 00:00:00,35 days 00:00:00,37.647059
1:2,34615.0759,40083.4655,246.150759,-80.116969,-25.144468,236 days 00:00:00,50 days 00:00:00,40.697674
1:4,72209.8689,81383.2585,622.098689,-89.169987,-21.552632,487 days 00:00:00,40 days 00:00:00,33.918129
